In [1]:
!nvidia-smi

Fri Mar 31 13:14:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:1B:00.0 Off |                  Off |
|  0%   34C    P8    28W / 460W |      8MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:1C:00.0 Off |                  N/A |
| 27%   

In [2]:
!pip install torch
!pip install transformers

In [3]:
import numpy as np
import torch

from transformers import T5ForConditionalGeneration, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoConfig


from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
from transformers.file_utils import cached_property
from typing import Tuple
from sklearn.model_selection import train_test_split
import gc
from tqdm.auto import tqdm, trange


import pandas as pd
from sklearn.utils import shuffle

from collections import Counter
from tqdm import tqdm

# model

In [4]:
from get_cuda_device import _get_cuda_command
import os

get_cuda_command('cuda:2')

ImportError: cannot import name '_get_cuda_command' from 'get_cuda_device' (/home/jovyan/work/few_shot/get_cuda_device.py)

In [8]:
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [4]:
!nvidia-smi

Sat Mar 25 17:21:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:1B:00.0 Off |                  Off |
|  0%   32C    P8    26W / 460W |   2609MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:1C:00.0 Off |                  N/A |
| 27%   

In [9]:
if torch.cuda.is_available():
    device='cuda'
    print('GPU')
else:
    device='cpu'
    print('CPU')
    
    
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

GPU


In [5]:
from transformers import TrainingArguments, Trainer

model_checkpoint = "t5-large"
model2 = T5ForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

NameError: name 'device' is not defined

In [ ]:
class PairsDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        assert idx <= len(self.x['input_ids']), (idx, len(self.x['input_ids']))
        item = {key: val[idx] for key, val in self.x.items()}
        item['decoder_attention_mask'] = self.y['attention_mask'][idx]
        item['labels'] = self.y['input_ids'][idx]
        return item
    
    @property
    def n(self):
        return len(self.x['input_ids'])

    def __len__(self):
        return self.n

In [12]:
from typing import List, Dict, Union

class DataCollatorWithPadding:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.tokenizer.pad(
            features,
            padding=True,
        )
        ybatch = self.tokenizer.pad(
            {'input_ids': batch['labels'], 'attention_mask': batch['decoder_attention_mask']},
            padding=True,
        ) 
        batch['labels'] = ybatch['input_ids']
        batch['decoder_attention_mask'] = ybatch['attention_mask']
        
        return {k: torch.tensor(v) for k, v in batch.items()}

In [13]:
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    
cleanup()

In [14]:
def evaluate_model(model, test_dataloader):
    num = 0
    den = 0

    for batch in test_dataloader:
        with torch.no_grad():
            loss = model(**{k: v.to(model.device) for k, v in batch.items()}).loss
            num += len(batch) * loss.item()
            den += len(batch)
    val_loss = num / den
    return val_loss

# **Read data SemEval2018-Task9**

In [8]:
import os

path = os.getcwd().replace('few_shot', '')

In [9]:
path_data_en = path+"SemEval2018-Task9/training/data/1A.english.training.data.txt"
path_gold_en = path+"SemEval2018-Task9/training/gold/1A.english.training.gold.txt"

train_data_en_data = pd.read_csv(path_data_en, header=None, sep="\t", names=['term', 'relation'])
train_gold_en_data = pd.read_csv(path_gold_en, header=None, names=['hypernym'])

train_data_en_data.head()

,term,relation
0,blackfly,Concept
1,Turonian,Entity
2,abhorrence,Concept
3,tropical storm,Concept
4,militarization,Concept


In [10]:
path_test_data_en = path+"SemEval2018-Task9/test/data/1A.english.test.data.txt"
path_test_gold_en = path+"SemEval2018-Task9/test/gold/1A.english.test.gold.txt"

test_data_en_data = pd.read_csv(path_test_data_en, header=None, sep="\t", names=['term', 'relation'])
test_gold_en_data = pd.read_csv(path_test_gold_en, header=None, names=['hypernym'])

In [11]:
pattern = 'My favorite [PARENT] is'

In [12]:
def hearest_preprocessing(train_features, train_target, test_features, test_target, hearst_pattern=pattern):
    hearst_pattern = hearst_pattern.replace('[PARENT]', '<extra_id_0>')
    
    prefix = 'fill | '
    prefix=''
        
    train_data_en = train_features.copy()
    train_data_en = prefix + hearst_pattern + ' ' + train_data_en_data.term +  ' </s>'
    print(train_data_en.head())

    train_gold_en = train_target.copy()
    train_gold_en = train_gold_en.hypernym.str.split('\t').str.join(', ')
    print(train_gold_en.head())
    
    test_data_en = test_features.copy()
    test_data_en = prefix + hearst_pattern + ' ' + test_data_en.term +  ' </s>'
    print(test_data_en.head())

    test_gold_en = test_target.copy()
    test_gold_en = test_gold_en.hypernym.str.split('\t').str.join(', ')
    print(test_gold_en.head())
    
    return train_data_en, train_gold_en, test_data_en, test_gold_en

In [13]:
train_data_en, train_gold_en, test_data_en, test_gold_en = hearest_preprocessing(train_data_en_data, 
                                                                                 train_gold_en_data, 
                                                                                 test_data_en_data, 
                                                                                 test_gold_en_data)

0          My favorite <extra_id_0> is blackfly </s>
1          My favorite <extra_id_0> is Turonian </s>
2        My favorite <extra_id_0> is abhorrence </s>
3    My favorite <extra_id_0> is tropical storm </s>
4    My favorite <extra_id_0> is militarization </s>
Name: term, dtype: object
0                           homopterous insect, insect
1    technical specification, geologic timescale, p...
2                      distaste, hatred, hate, disgust
3    atmosphere, windstorm, violent storm, air curr...
4                                       social control
Name: hypernym, dtype: object
0    My favorite <extra_id_0> is maliciousness </s>
1          My favorite <extra_id_0> is buckler </s>
2        My favorite <extra_id_0> is spelunker </s>
3     My favorite <extra_id_0> is quo warranto </s>
4     My favorite <extra_id_0> is Jeff Francis </s>
Name: term, dtype: object
0       malevolence, distaste, hatred, hate, malignity
1                                           body armor
2       

In [14]:
def _filter(output, end_token='<extra_id_1>'):
        # The first token is <unk> (inidex at 0) and the second token is <extra_id_0> (indexed at 32099)
        _txt = tokenizer.decode(output[2:], skip_special_tokens=False, clean_up_tokenization_spaces=False)
        if end_token in _txt:
            _end_token_index = _txt.index(end_token)
            return _txt[:_end_token_index]
        else:
            return _txt

        
def predict_token(text):
    
    encoded = tokenizer.encode_plus(text, add_special_tokens=False, return_tensors='pt')
    input_ids = encoded['input_ids'].to(device)

    # Generaing 20 sequences with maximum length set to 10
    outputs = model.generate(input_ids=input_ids, 
                              num_beams=20, 
                              num_return_sequences=20,
                              max_length=10)

    _0_index = text.index('<extra_id_0>')
    _result_prefix = text[:_0_index]
    _result_suffix = text[_0_index+12:]  # 12 is the length of <extra_id_0>
    
    results = list(map(_filter, outputs))
    results = [test_string.replace(',' , '')for test_string in results]
    
    return results

def generate_sample(train_data_en, train_gold_en):
    indexes = np.random.randint(0, len(train_gold_en), size=3)

    selected_hypernyms = train_gold_en[indexes].str.split(', ').tolist()
    random_selected_hypernym = [np.random.choice(target_hypernym) for target_hypernym in selected_hypernyms]
    
    random_selected_samples = train_data_en[indexes].str.replace('</s>', '').tolist()

    return [sample.replace('<extra_id_0>', target_sample) for sample, target_sample in zip(random_selected_samples, 
                                                                                    random_selected_hypernym)]

In [15]:
generate_sample(train_data_en, train_gold_en)

['My favorite granular material is greensand ',
 'My favorite person is Sam Rainsy ',
 'My favorite person is Kathryn Werdegar ']

In [20]:
def predict(test_data_en, test_gold_en):
    
#   make predictions for each hyponyms
    test_pred_en=[]
    for text in tqdm(test_data_en.tolist()):
        
#   few_shot procedure

        # print(text)
        few_shot = generate_sample(train_data_en, train_gold_en)
        few_shot.append(text)
        text = '.\n'.join(few_shot)
        print(text)
        
        
        # pred_masked_token = predict_token(text)
        test_pred_en.append('\t'.join(pred_masked_token))
            
#   make txt format
    name  = 'pred_few_shot.txt'

    test_pred_en_df = pd.DataFrame(test_pred_en)
    test_pred_en_df.to_csv(name, header=None, index=None)

In [21]:
predict(test_data_en, test_gold_en)

  0%|                                                                                          | 0/1500 [00:00<?, ?it/s]

My favorite difference of opinion is warfare .
My favorite military rank is Richard Barnes Mason .
My favorite pant is gabardine .
My favorite <extra_id_0> is maliciousness </s>


NameError: name 'pred_masked_token' is not defined

In [21]:
from contextlib import redirect_stdout
import io

def answers(str_ans, name_of_pattern):
    str_ans += name_of_pattern
    columns_name = []
    values = []
    for ind, metrics in enumerate(_std_out.split('\r\n')):
        
        if ind == 6:
            _name = 'number of samples in few-shot'
            number = name_of_pattern
        else:
            _name, number = metrics.split(' ')
            number = round(float(number), 5)
            _name = _name[:-1]
        
        columns_name.append(_name)
        values.append([number])
        
    
        
    df = pd.DataFrame(values).T
    df.columns = columns_name
    df.set_index('number of samples in few-shot', inplace=True)
    return df

# Prediction

In [26]:
%autosave 30

Autosaving every 30 seconds


In [27]:
from tqdm import tqdm
from IPython.display import clear_output

In [28]:
few_shot_table = []
number_of_few_shots = np.round(np.linspace(1, 10, 5).tolist() + np.linspace(1, 100, 5).tolist()[1:]).astype(int)


for _number_few_shot in tqdm(number_of_few_shots):
    
#   Data processing
    train_data_en, train_gold_en, test_data_en, test_gold_en = hearest_preprocessing(train_data_en_data,
                                                                                     train_gold_en_data, 
                                                                                     test_data_en_data, 
                                                                                     test_gold_en_data)
#   Predict
    predict(test_data_en, test_gold_en)
    
#   Metrics
    
    f = io.StringIO()
    with redirect_stdout(f):
        !python /home/jovyan/work/debuged_task9-scorer.py /home/jovyan/work/SemEval2018-Task9/test/gold/1A.english.test.gold.txt pred_few_shot.txt


    _std_out = f.getvalue()


    _std_out = _std_out  + str(_number_few_shot)
    
    _table = answers(_std_out, str(_number_few_shot))
    few_shot_table.append(_table)


    df = pd.concat(few_shot_table)
    df.to_csv('few_shot_result_table.csv')
    clear_output(wait=True)

100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [1:20:52<00:00, 485.27s/it]


In [6]:
def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')

def highlight_min(s, props=''):
    return np.where(s == np.nanmin(s.values), props, '')

In [7]:
results_no_few_shot_learning = pd.read_csv('/home/jovyan/work/hearst_patterns/hearst_patterns_result_table.csv')


mask_ = results_no_few_shot_learning['name of hearst pattern']=='My favorite [PARENT] is'
no_few = results_no_few_shot_learning[mask_]

no_few.drop('name of hearst pattern', axis=1, inplace=True)

no_few['number of few_shot samples'] = 0
no_few.set_index('number of few_shot samples', inplace=True)

/tmp/ipykernel_98/18661040.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_few.drop('name of hearst pattern', axis=1, inplace=True)
/tmp/ipykernel_98/18661040.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_few['number of few_shot samples'] = 0


In [8]:
metrics

,MRR,MAP,P@1,P@3,P@5,P@15
number of few_shot samples,,,,,,
1,0.16482,0.08666,0.11533,0.08822,0.08459,0.08231
3,0.16003,0.08604,0.11267,0.08800,0.08550,0.08078
6,0.17638,0.09367,0.12667,0.09667,0.09104,0.08855
8,0.16208,0.08656,0.10867,0.08778,0.08652,0.08280
10,0.16923,0.08901,0.11267,0.09167,0.08686,0.08476
26,0.15929,0.08108,0.11533,0.08556,0.07822,0.07541
50,0.16053,0.08811,0.10867,0.08900,0.08684,0.08510
75,0.16932,0.09111,0.11733,0.09544,0.09151,0.08539
100,0.16583,0.08824,0.11533,0.09000,0.08759,0.08376


In [10]:
metrics = pd.read_csv('few_shot_result_table.csv', index_col='number of few_shot samples')


metrics = pd.concat([no_few, metrics])



metrics_table = metrics.style.apply(highlight_max, props='color:white; background-color:#1FC29D', axis=0).apply(highlight_min, props='color:white; background-color:#FF5555', axis=0)
metrics_table

,MRR,MAP,P@1,P@3,P@5,P@15
number of few_shot samples,,,,,,
0,0.180120,0.108980,0.114670,0.095330,0.094000,0.121550
1,0.164820,0.086660,0.115330,0.088220,0.084590,0.082310
3,0.160030,0.086040,0.112670,0.088000,0.085500,0.080780
6,0.176380,0.093670,0.126670,0.096670,0.091040,0.088550
8,0.162080,0.086560,0.108670,0.087780,0.086520,0.082800
10,0.169230,0.089010,0.112670,0.091670,0.086860,0.084760
26,0.159290,0.081080,0.115330,0.085560,0.078220,0.075410
50,0.160530,0.088110,0.108670,0.089000,0.086840,0.085100
75,0.169320,0.091110,0.117330,0.095440,0.091510,0.085390
